**Task**: It is your job to predict the sales price for each house. For each Id in the test set, you must predict the value of the SalePrice variable. 

**Evaluation**: Submissions are evaluated on Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price.

This notebook has been created after studying [this notebook.](https://www.kaggle.com/ankitverma2010/house-prices-prediction-beginner-to-advanced#Exploratory-Data-Analysis)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Import libraries

In [ ]:
# Python imports
import os

# Maths and data imports
import numpy as np
import pandas as pd

# Plot imports
import seaborn as sns
import matplotlib.pyplot as plt

# ML modeling imports
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error as mse
from xgboost import XGBRegressor

In [ ]:
%matplotlib inline
sns.set()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv', index_col='Id')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv', index_col='Id')

In [ ]:
train_df = train.copy()
test_df = test.copy()

## Data Preprocessing

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
# MSSubClass, OverallQual and OverallCond and categorical variable, lets convert it into one.
train_df['MSSubClass'] = train_df['MSSubClass'].astype('object')
train_df['OverallQual'] = train_df['OverallQual'].astype('object')
train_df['OverallCond'] = train_df['OverallCond'].astype('object')

test_df['MSSubClass'] = test_df['MSSubClass'].astype('object')
test_df['OverallQual'] = test_df['OverallQual'].astype('object')
test_df['OverallCond'] = test_df['OverallCond'].astype('object')

Let's now seperate the categorical and numerical columns

In [ ]:
def get_var_dtype_list(df):
    cat_cols = []
    num_cols = []

    for col in df.columns:
        if df[col].dtypes == 'object':
            cat_cols.append(col)
        else:
            num_cols.append(col)
    return (cat_cols, num_cols)

In [ ]:
cat_cols_train, num_cols_train = get_var_dtype_list(train_df)

In [ ]:
cat_cols_test, num_cols_test = get_var_dtype_list(test_df)

Let's take log of SalePrice as our evaluation metric is log RMS value.

In [ ]:
train_df['SalePrice'] = np.log(train_df['SalePrice'])

## EDA

In EDA we will analyse data to see if there is any - 
1. Skewness in features
2. Check for missing values and ouliers and fix them.
3. Check the variability of different features and scale them.
4. Check for multicolinearity among multiple exploratory variables (features).
5. Check if response variable is correlated to any/may exploratory variable(s).
6. Analyse the target/response variable.

### Lets check distribution fo continuous columns

In [ ]:
fig, axs = plt.subplots(len(num_cols_train)//5, 5, figsize=(30, 30))
for col, ax in zip(num_cols_train[:-1], axs.flatten()):
    sns.distplot(train_df[col], ax=ax)
    ax.set_title(col)
    ax.set_xlabel('')
    ax.set_ylabel('')
plt.show()

**Conclusion**

1. There is  large variation in the scale of each continuous variables, hence, we need to scale them.
2. Features such as YearBuilt and GarageYrBlt are left skewed, indicating that more houses were build in the later years hence, more garages were also built then. So, we might want to leave such variable for outlier check.
3. Exploratory varibales - 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'LowQualFinSF', 'BsmtHalfBath', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal' are heavily right skewed.

Let's further explore these variables.

In [ ]:
right_skewed = ['LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                'LowQualFinSF', 'BsmtHalfBath', 'WoodDeckSF', 'OpenPorchSF', 
                'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']

In [ ]:
r_skew_desc = train_df[right_skewed].describe().T
r_skew_desc['coef_of_var'] = r_skew_desc['std']/r_skew_desc['mean']

r_skew_desc

We can see a huge variation in lot of features. For now we can drop the columns with coefficient of variation > 3.

As the number of missing values is zero or near to zero in most of the features we would like to further analyse them (maybe take log to reduce the right skewness) and embrace the variability.

But for now lets got with dropping them off, and compare the effects in the next iteration/followup.

In [ ]:
r_skew_desc[r_skew_desc['coef_of_var']>3].T.columns

In [ ]:
drop_skew_cols = ['BsmtFinSF2', 'LowQualFinSF', 'BsmtHalfBath', '3SsnPorch',
                  'ScreenPorch', 'PoolArea', 'MiscVal']

train_df.drop(drop_skew_cols, axis=1, inplace=True)
test_df.drop(drop_skew_cols, axis=1, inplace=True)

In [ ]:
train_df.columns

Let's update out list of numerical columns list.

In [ ]:
cat_cols_train, num_cols_train = get_var_dtype_list(train_df)
cat_cols_test, num_cols_test = get_var_dtype_list(test_df)

### Let check and impute missing values in continuous variables

In [ ]:
def get_missing_stats(df, col_list, threshold=0):
    total = len(df)
    for col in col_list:
        null = df[col].isnull().sum()
        if null > 0 and null/total >= threshold:
            print(col)
            if df[col].dtypes == 'object':
                print(df[col].value_counts())
            print(f'Missing values: {null} of {total}')
            print(f'Percent missing values: {round((null*100)/total, 2)}%\n')

In [ ]:
get_missing_stats(train_df, num_cols_train)

In [ ]:
get_missing_stats(test_df, num_cols_test)

It does not seem to be extreme case of missing values so, we can simply replace by the median (as the data seem to be highly variable and median is a robust metric) of training set in both the training and test set.

In [ ]:
imputer = SimpleImputer(strategy='median')
train_df[num_cols_train[:-1]] = imputer.fit_transform(train_df[num_cols_train[:-1]])
test_df[num_cols_test] = imputer.transform(test_df[num_cols_test])

In [ ]:
get_missing_stats(train_df, num_cols_train)

In [ ]:
get_missing_stats(test_df, num_cols_test)

So, the missing values for numerical columns have disappeared. Now, lets hit the categorical columns.

### Now, lets check and impute missing values of categorical columns.

In [ ]:
get_missing_stats(train_df, cat_cols_train)

In [ ]:
get_missing_stats(test_df, cat_cols_test)

Some columns seet to have quite a number of missing values. Lets further analyse them using barplot.

In [ ]:
fig, axs = plt.subplots(len(cat_cols_train)//5, 5, figsize=(30, 40))
for col, ax in zip(cat_cols_train, axs.flatten()):
    y = train_df[col].value_counts()
    ax.bar(y.index, y.values)
    ax.set_title(col)
    ax.set_xlabel('')
    ax.set_ylabel('')
    for tick in ax.get_xticklabels():
        tick.set_rotation(35)
plt.show()

Let's just analyse the columns with missing frequency greater than 40%.

In [ ]:
get_missing_stats(train_df, cat_cols_train, 0.4)

In [ ]:
get_missing_stats(test_df, cat_cols_test, 0.4)

We can infer two cases from above:
1. The houses did not have these features
2. The houses did have these features but were not reported.

However, if we think then it seems feasible to assume that all the houses surveyed did not have all the 80 features. So, for now we can impute the missing values (with freq. > 40%) with N.A, where as missing values of columns with freq < 40% with most frequent value.

I think it will be safe to assume and hypothesise that maybe the localities in which the houses were surveyed did not have the above features, thus introducing selective bias in data collection. Although, we can not be certain of it.

In [ ]:
train_df.drop(['PoolQC'], axis=1, inplace=True)
test_df.drop(['PoolQC'], axis=1, inplace=True)

In [ ]:
na_cols = ['Alley','FireplaceQu','Fence','MiscFeature']

na_imputer = SimpleImputer(strategy='constant', fill_value='N.A')
train_df[na_cols] = na_imputer.fit_transform(train_df[na_cols])
test_df[na_cols] = na_imputer.transform(test_df[na_cols])

In [ ]:
cat_cols_train, num_cols_train = get_var_dtype_list(train_df)
cat_cols_test, num_cols_test = get_var_dtype_list(test_df)

In [ ]:
mf_imputer = SimpleImputer(strategy='most_frequent')
train_df[cat_cols_train] = mf_imputer.fit_transform(train_df[cat_cols_train])
test_df[cat_cols_test] = mf_imputer.transform(test_df[cat_cols_test])

In [ ]:
get_missing_stats(train_df, cat_cols_train)

In [ ]:
get_missing_stats(test_df, cat_cols_test)

Finally, we have dealt with missing values. Now, lets check for multicolinearity among exploratory variables.

In [ ]:
plt.figure(figsize=(20, 20))
sns.heatmap(train_df.drop(['SalePrice'], axis=1).corr(), annot=True)
plt.show()

In [ ]:
cor = train_df.drop(['SalePrice'], axis=1).corr()
for i, col in enumerate(cor.columns):
    for row in cor.index[i+1:]:
        if col != row and cor[col][row] > 0.7:
            print(f'({row}, {col}): {cor[col][row]}')

Hmm...From the overwhelming plots above we can see that few varibles are highly correlated, and it makes sense for them to be correlated. Hence, for now we might want to keep them.

### Scale variables

In [ ]:
scaler = StandardScaler()
train_df[num_cols_train[:-1]] = scaler.fit_transform(train_df[num_cols_train[:-1]])
test_df[num_cols_test] = scaler.transform(test_df[num_cols_test])

### One-hot encoding of categorical columns

For now, let's simply one hot encode the categorical columns.

In [ ]:
cat_cols_train

In [ ]:
cat_cols_train, num_cols_train = get_var_dtype_list(train_df)
cat_cols_test, num_cols_test = get_var_dtype_list(test_df)

In [ ]:
train_df = pd.get_dummies(train_df, drop_first=True, columns=cat_cols_train)
test_df = pd.get_dummies(test_df, drop_first=True, columns=cat_cols_test)

In [ ]:
train_df.head()

### Check train and test for compatability

In [ ]:
# check if both train and test contain same columns
train_df.columns

In [ ]:
test_df.columns

In [ ]:
compat_list = list(set(train_df.columns).intersection(test_df.columns))

In [ ]:
len(compat_list)

In [ ]:
train_X, y = train_df[compat_list], train_df['SalePrice']
test_X = test_df[compat_list]

In [ ]:
(train_X.columns == test_X.columns).sum()

## Train Test split

In [ ]:
train_x, valid_x, train_y, valid_y = train_test_split(train_X, y, test_size=0.2, random_state=42)

## Model

In [ ]:
linear_regressor = LinearRegression()
linear_regressor.fit(train_x, train_y)
preds = linear_regressor.predict(valid_x)
mse(valid_y, preds, squared=False)

In [ ]:
models = {
    'RFR': RandomForestRegressor,
    'ADR': AdaBoostRegressor,
    'XGB': XGBRegressor
}

In [ ]:
def fit_model(name, model, train_ds, valid_ds):
    X, y = train_ds
    X_val, y_val = valid_ds
    
    model.fit(X, y)
    y_hat = model.predict(X)
    y_hat_val = model.predict(X_val)
    
    mse_ = mse(y, y_hat, squared=False)
    mse_val = mse(y_val, y_hat_val, squared=False)
    
    print(f'Model: {name}, Train MSE: {mse_}, Val MSE: {mse_val}')

In [ ]:
n_est = [10, 25, 50, 100, 125]
for i in range(len(n_est)):
    print(f'n_estimators: {n_est[i]}')
    for name, model in models.items():
        model = model(n_estimators=n_est[i])
        fit_model(name, model, (train_x, train_y), (valid_x, valid_y))
    print('-'*20)

`n_estimators=25` seems like a sweet spot for `AdaBoostRegressor` and `XGBRegressor`. Lets ensemble te ensembles

In [ ]:
abr = AdaBoostRegressor(n_estimators=25)
xgr = XGBRegressor(n_estimators=25)

abr.fit(train_x, train_y)
xgr.fit(train_x, train_y)

p1 = abr.predict(valid_x)
p2 = xgr.predict(valid_x)
p3 = linear_regressor.predict(valid_x)

m1 = mse(valid_y, p1, squared=False)
m2 = mse(valid_y, p2, squared=False)
m3 = mse(valid_y, p3, squared=False)
m4 = mse(valid_y, (p1+p2)/2, squared=False)
m5 = mse(valid_y, (p1+p2+p3)/3, squared=False)

print(f'Ensemble MSE: \nABR: {m1}\nXGR: {m2}\nLinear: {m3}\nABR+XGR: {m4}\nABR+XGR+Linear: {m5}')

Let's try them all, 
* linear regressor
* Ensemble of AdaBoost and XGBoost
* Ensemble of all of the above

In [ ]:
pred_1 = linear_regressor.predict(test_X)
pred_2 = (abr.predict(test_X) + xgr.predict(test_X))/2
pred_3 = (2*pred_2 + pred_1)/3

In [ ]:
def create_submission(preds, name='submission.csv'):
    preds = np.expm1(preds)
    submission = pd.DataFrame({'Id': test_X.index, 'SalePrice': preds})
    submission.to_csv(name, index=False)
    return submission

In [ ]:
create_submission(pred_1, 'submission1.csv')

In [ ]:
create_submission(pred_2, 'submission2.csv')

In [ ]:
create_submission(pred_3, 'submission3.csv')

## References:

https://www.kaggle.com/ankitverma2010/house-prices-prediction-beginner-to-advanced#Exploratory-Data-Analysis